In [1]:
import pandas as pd
import numpy as np
import requests
import sqlite3
from plyer import notification 

##### Envio de notificação de ERRO

In [3]:
def alerta(nivel, url, etapa, erro=""):

    msg = f"Falha no carregamento da base {url} na etapa {etapa}.\n{erro}"


    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel  == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nivel",nivel,"não disponível!")

    notification.notify(
            title=title,
            message=msg,
            app_name='alerta',
            timeout=10
        )

##### 1 - Realiza get na api cidade

In [4]:
def get_base_localidades():
    url = "https://brasilapi.com.br/api/cptec/v1/cidade"

    try:
        response = requests.get(url)
        data_json = response.json()
    except Exception as e:
        alerta(nivel = 3, 
               url = url, 
               etapa = (f"REQUEST, {response}"), 
               erro=e)


    df_localidades = pd.DataFrame(data_json)

    df_localidades = df_localidades.rename(columns={'nome':'cidade','id':'id_cidade'})

    return df_localidades

##### 1.1 - Print das localidades

In [5]:
get_base_localidades()

,cidade,id_cidade,estado
0,São Benedito,4750,CE
1,São Benedito do Rio Preto,4751,MA
2,São Benedito do Sul,4752,PE
3,São Bento,4753,MA
4,São Bento,4754,PB
5,São Bento Abade,4755,MG
6,São Bento de Pombal,4756,PB
7,São Bento do Norte,4757,RN
8,São Bento do Sapucaí,4758,SP
9,São Bento do Sul,4759,SC


##### 2 - Realiza get na api previsao do tempo atraves do id_cidade do item 1

In [6]:
#Gera a previsao do tempo para cada cidade do item 1 atraves do id_cidade
def get_previsao_tempo():
    cidades = get_base_localidades()
    list_previsao = []

    #Utiliza o id_cidade para gerar a previsao do tempo para cada cidade
    for id_cityCode in cidades.id_cidade:
        try:
            url = (f'https://brasilapi.com.br/api/cptec/v1/clima/previsao/' + str(id_cityCode)) 
            response_previsao = requests.get(url)
            data_jason = response_previsao.json()
            list_previsao.append (data_jason)
        except Exception as e:
            alerta(nivel = 3, 
                url = url, 
                etapa = (f"REQUEST, {response_previsao}"), 
                erro=e)
            break

    
    return list_previsao

##### 2.1 - Print da previsão do tempo do dia + 1

In [7]:
get_previsao_tempo()

[{'cidade': 'São Benedito',
  'estado': 'CE',
  'atualizado_em': '2024-04-30',
  'clima': [{'data': '2024-05-01',
    'condicao': 'pn',
    'condicao_desc': 'Parcialmente Nublado',
    'min': 21,
    'max': 29,
    'indice_uv': 11}]},
 {'cidade': 'São Benedito do Rio Preto',
  'estado': 'MA',
  'atualizado_em': '2024-04-30',
  'clima': [{'data': '2024-05-01',
    'condicao': 'ci',
    'condicao_desc': 'Chuvas Isoladas',
    'min': 23,
    'max': 30,
    'indice_uv': 11}]},
 {'cidade': 'São Benedito do Sul',
  'estado': 'PE',
  'atualizado_em': '2024-04-30',
  'clima': [{'data': '2024-05-01',
    'condicao': 'ci',
    'condicao_desc': 'Chuvas Isoladas',
    'min': 23,
    'max': 30,
    'indice_uv': 10}]},
 {'cidade': 'São Bento',
  'estado': 'MA',
  'atualizado_em': '2024-04-30',
  'clima': [{'data': '2024-05-01',
    'condicao': 'ci',
    'condicao_desc': 'Chuvas Isoladas',
    'min': 23,
    'max': 32,
    'indice_uv': 12}]},
 {'cidade': 'São Bento',
  'estado': 'PB',
  'atualizado_e

##### 3 - Gera a previsao do tempo para cada cidade do item 1 atraves do id_cidade, para os próximos 6 dias

In [8]:

def get_previsao_tempo_v2():
    cidades = get_base_localidades()
    list_previsao = []

            #Utiliza o id_cidade para gerar a previsao do tempo para cada cidade
    for id_cityCode in cidades.id_cidade:
            try:
                url = (f'https://brasilapi.com.br/api/cptec/v1/clima/previsao/' + str(id_cityCode) +'/6') 
                response_previsao = requests.get(url)
                data_jason = response_previsao.json()
                list_previsao.append (data_jason)
            except Exception as e:
                alerta(nivel = 3, 
                        url = url, 
                        etapa = (f"REQUEST, {response_previsao}"), 
                        erro=e)
                break
    df = pd.DataFrame(list_previsao)
    clima = df.clima
    return clima

##### 3.1 - Print no formato json da data, condições do tempo, temperatura minima e temperatura máxima dos próximos 5 dias do item 1

In [10]:
get_previsao_tempo_v2()

0     [{'data': '2024-05-01', 'condicao': 'pn', 'con...
1     [{'data': '2024-05-01', 'condicao': 'ci', 'con...
2     [{'data': '2024-05-01', 'condicao': 'ci', 'con...
3     [{'data': '2024-05-01', 'condicao': 'ci', 'con...
4     [{'data': '2024-05-01', 'condicao': 'pn', 'con...
5     [{'data': '2024-05-01', 'condicao': 'pn', 'con...
6     [{'data': '2024-05-01', 'condicao': 'ci', 'con...
7     [{'data': '2024-05-01', 'condicao': 'ci', 'con...
8     [{'data': '2024-05-01', 'condicao': 'pn', 'con...
9     [{'data': '2024-05-01', 'condicao': 'pn', 'con...
10    [{'data': '2024-05-01', 'condicao': 'pn', 'con...
11    [{'data': '2024-05-01', 'condicao': 'ci', 'con...
12    [{'data': '2024-05-01', 'condicao': 'ci', 'con...
13    [{'data': '2024-05-01', 'condicao': 'pn', 'con...
14    [{'data': '2024-05-01', 'condicao': 'ci', 'con...
15    [{'data': '2024-05-01', 'condicao': 'pn', 'con...
16    [{'data': '2024-05-01', 'condicao': 'pn', 'con...
17    [{'data': '2024-05-01', 'condicao': 'c', '

##### 3.2 - Gera pandas DataFrame da data, condições do tempo, temperatura minima e temperatura máxima dos próximos 5 dias do item 3.1

In [11]:
def separa_clima_cinco_dias():
    clima = get_previsao_tempo_v2()
    
    list_data = []
    lista1 =[]
    lista2 =[]
    lista3 =[]
    lista4 =[]
    lista5 =[]
    lista6 =[]
    list_condicoes = []
    clima1 = []
    clima2 = []
    clima3 = []
    clima4 = []
    clima5 = []
    clima6 = []
    list_temp_min = []
    min1 = []
    min2 = []
    min3 = []
    min4 = []
    min5 = []
    min6 = []
    list_temp_max = []
    max1 = []
    max2 = []
    max3 = []
    max4 = []
    max5 = []
    max6 = []

    for climas in clima:
        for dados in climas:
            list_data.append(dados.get('data'))
            list_condicoes.append(dados.get('condicao_desc'))
            list_temp_min.append(dados.get('min'))
            list_temp_max.append(dados.get('max'))

    i=0
    while i < len(list_data):
        i = i + 1
        lista1.append(list_data[i])
        df_data1 = pd.DataFrame(lista1, columns=['data1'])
        clima1.append(list_condicoes[i])
        df_clima1 = pd.DataFrame(clima1, columns=['clima1']) 
        min1.append(list_temp_min[i])
        df_min1 = pd.DataFrame(min1, columns=['min1']) 
        max1.append(list_temp_max[i])
        df_max1 = pd.DataFrame(max1, columns=['max1'])
        i = i + 1
        
        lista2.append(list_data[i])
        df_data2 = pd.DataFrame(lista2, columns=['data2'])
        clima2.append(list_condicoes[i])
        df_clima2 = pd.DataFrame(clima2, columns=['clima2'])
        min2.append(list_temp_min[i])
        df_min2 = pd.DataFrame(min2, columns=['min2']) 
        max2.append(list_temp_max[i])
        df_max2 = pd.DataFrame(max2, columns=['max2'])
        i = i + 1
        
        lista3.append(list_data[i])
        df_data3 = pd.DataFrame(lista3, columns=['data3'])
        clima3.append(list_condicoes[i])
        df_clima3 = pd.DataFrame(clima3, columns=['clima3'])
        min3.append(list_temp_min[i])
        df_min3 = pd.DataFrame(min3, columns=['min3']) 
        max3.append(list_temp_max[i])
        df_max3 = pd.DataFrame(max3, columns=['max3'])
        i = i + 1
        
        lista4.append(list_data[i])
        df_data4 = pd.DataFrame(lista4, columns=['data4'])
        clima4.append(list_condicoes[i])
        df_clima4 = pd.DataFrame(clima4, columns=['clima4'])
        min4.append(list_temp_min[i])
        df_min4 = pd.DataFrame(min4, columns=['min4']) 
        max4.append(list_temp_max[i])
        df_max4 = pd.DataFrame(max4, columns=['max4'])
        i = i + 1

        lista5.append(list_data[i])
        df_data5 = pd.DataFrame(lista5, columns=['data5'])
        clima5.append(list_condicoes[i])
        df_clima5 = pd.DataFrame(clima5, columns=['clima5'])
        min5.append(list_temp_min[i])
        df_min5 = pd.DataFrame(min5, columns=['min5']) 
        max5.append(list_temp_max[i])
        df_max5 = pd.DataFrame(max5, columns=['max5'])
        i = i + 1   

        data_concat = pd.concat([df_data1, df_clima1, df_min1, df_max1,
                                df_data2, df_clima2, df_min2, df_max2,
                                df_data3, df_clima3, df_min3, df_max3,
                                df_data4, df_clima4, df_min4, df_max4,
                                df_data5, df_clima5, df_min5, df_max5], axis=1)
                                #df_data6, df_clima6, df_min6, df_max6
    clima_concat = data_concat

    return clima_concat

##### 3.3 - Print do intem 3.2

In [12]:
separa_clima_cinco_dias()

,data1,clima1,min1,max1,data2,clima2,min2,max2,data3,clima3,min3,max3,data4,clima4,min4,max4,data5,clima5,min5,max5
0,2024-05-02,Parcialmente Nublado,21,29,2024-05-03,Parcialmente Nublado,21,29,2024-05-04,Parcialmente Nublado,21,30,2024-05-05,Chuvas Isoladas,22,29,2024-05-06,Parcialmente Nublado,22,30
1,2024-05-02,Chuva,24,32,2024-05-03,Chuva,26,32,2024-05-04,Parcialmente Nublado,25,34,2024-05-05,Chuva,25,35,2024-05-06,Chuva,26,32
2,2024-05-02,Chuva,23,28,2024-05-03,Chuva,22,27,2024-05-04,Chuvas Isoladas,22,30,2024-05-05,Chuva,24,30,2024-05-06,Chuva,23,29
3,2024-05-02,Chuva,24,28,2024-05-03,Chuva,23,28,2024-05-04,Chuva,25,31,2024-05-05,Chuva,25,29,2024-05-06,Chuva,26,28
4,2024-05-02,Parcialmente Nublado,23,36,2024-05-03,Chuva,24,35,2024-05-04,Chuvas Isoladas,23,33,2024-05-05,Parcialmente Nublado,23,37,2024-05-06,Parcialmente Nublado,22,37
5,2024-05-02,Parcialmente Nublado,17,29,2024-05-03,Parcialmente Nublado,17,30,2024-05-04,Parcialmente Nublado,18,30,2024-05-05,Parcialmente Nublado,16,30,2024-05-06,Parcialmente Nublado,17,31
6,2024-05-02,Chuvas Isoladas,23,35,2024-05-03,Chuva,23,35,2024-05-04,Chuvas Isoladas,22,32,2024-05-05,Parcialmente Nublado,22,35,2024-05-06,Parcialmente Nublado,23,37
7,2024-05-02,Chuva,24,32,2024-05-03,Parcialmente Nublado,24,32,2024-05-04,Parcialmente Nublado,24,33,2024-05-05,Chuvas Isoladas,24,33,2024-05-06,Chuvas Isoladas,24,32
8,2024-05-02,Parcialmente Nublado,16,26,2024-05-03,Parcialmente Nublado,15,27,2024-05-04,Parcialmente Nublado,15,27,2024-05-05,Parcialmente Nublado,15,27,2024-05-06,Parcialmente Nublado,15,28
9,2024-05-02,Parcialmente Nublado,17,29,2024-05-03,Parcialmente Nublado,19,29,2024-05-04,Parcialmente Nublado,16,29,2024-05-05,Parcialmente Nublado,18,30,2024-05-06,Parcialmente Nublado,16,30
